In [1]:
import matplotlib
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from matplotlib import style, colors
from matplotlib import gridspec

matplotlib.style.use('ggplot')
%matplotlib inline
matplotlib.rcParams['figure.figsize'] = (8, 6)

# make oncoprint for 124 patients

In [43]:
# get gene list
# f2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/genes_grouped_for_oncoprint.txt'
# f2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/chipseq/histogenes_genes_grouped_for_oncoprint.txt'
f2 = '/projects/trans_scratch/validations/workspace/szong/Cervical/mutsig2cv/118_patients/smgs_reviewed.txt'
cdf = pd.read_csv(f2, comment="#")
cdf.columns = ['genes']
genes = cdf.genes.tolist()
print(genes)

['MAPK1', 'CASP8', 'FBXW7', 'MLL2', 'ZC3H6', 'PCDHGA12', 'FAT1', 'ZNF750', 'SLC35G5', 'PCDHA9', 'PSPC1']


In [52]:
f1 ='/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/HIGH_MODERATE_INDEL_summary_with_normal_strelka_indels_only.tsv'
indf = pd.read_csv(f1, sep='\t')
indf.head(2)

,gene,num_patients_gene_level,num_INDELs_gene_level,chromosome,position,ref_base,alt_base,num_patients_INDEL_level,patient_ID,snp_ID,...,pileup_AltC,pileup_AF,strelka_n_Cov,strelka_n_RefC,strelka_n_AltC,strelka_n_AF,strelka_t_Cov,strelka_t_RefC,strelka_t_AltC,strelka_t_AF
0,PCNT,1,2,21,47849950,CAGG,C,1,HTMCP-03-06-02448,novel_snp,...,na,na,41,43,0,0.0,83,62,15,0.05
1,MLL3,29,6,7,151879476,CTG,C,1,HTMCP-03-06-02097,novel_snp,...,na,na,33,32,0,0.0,84,56,22,0.11


In [53]:
indf['impact_tmp'] = indf['snpeff_details'].apply(lambda x: x.split('(')[0])
indf = indf[['gene', 'patient_ID','impact_tmp']]
indf.head()
indf.shape

(827, 3)

In [56]:
indf.drop_duplicates(keep='first', inplace=True)

In [12]:
indf[indf.impact_tmp == 'frameshift_variant+stop_gained']
indf[indf.impact_tmp == 'frameshift_variant+splice_donor_variant+splice_region_variant+splice_region_variant+intron_variant']

,gene,patient_ID,impact_tmp
322,IL2RG,HTMCP-03-06-02157,frameshift_variant+splice_donor_variant+splice...


In [58]:
for i in sorted(indf.impact_tmp.unique()):
    print(i)

CODON_CHANGE_PLUS_CODON_DELETION
CODON_CHANGE_PLUS_CODON_INSERTION
CODON_DELETION
CODON_INSERTION
FRAME_SHIFT
SPLICE_SITE_ACCEPTOR
SPLICE_SITE_DONOR
STOP_GAINED
disruptive_inframe_deletion
frameshift_variant
frameshift_variant+splice_donor_variant+splice_region_variant+splice_region_variant+intron_variant
frameshift_variant+stop_gained
inframe_deletion
splice_donor_variant+disruptive_inframe_deletion+splice_region_variant+intron_variant
splice_donor_variant+intron_variant
splice_donor_variant+splice_region_variant+splice_region_variant+intron_variant+non_coding_exon_variant


In [42]:
# get meta track for example histology
f3 ='/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients/mutation_load_clinic.txt'
ddf = pd.read_csv(f3, sep='\t')
ddf = ddf[['patient', 'HIV_status', 'Putative_histology']]
edf = ddf.set_index('patient').T
edf.head(2)

patient,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02007,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02026,...,HTMCP-03-06-02424,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02434,HTMCP-03-06-02435,HTMCP-03-06-02437,HTMCP-03-06-02441,HTMCP-03-06-02442,HTMCP-03-06-02447,HTMCP-03-06-02448
HIV_status,Positive,Negative,Positive,Negative,Negative,Negative,Negative,Negative,Positive,Negative,...,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive
Putative_histology,Squamous,Squamous,Squamous,Squamous,Adenosquamous,Squamous,Squamous,Squamous,Squamous,Adeno,...,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous,Adeno


In [23]:
edf.loc['Putative_histology',].unique()

array(['Squamous', 'Adenosquamous', 'Adeno', 'Neuroendocrine'],
      dtype=object)

In [24]:
# get snvs
# f4 = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients/124_patients_selected_columns_new_filtered.txt'
f4 = '/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/HIGH_MODERATE_SNV_summary_with_normal_with_AF_filtered_somatic.tsv'
pdf = pd.read_csv(f4, sep='\t')
pdf.head(2)
pdf['impact_tmp'] = pdf['snpeff_details'].apply(lambda x: x.split('(')[0])
mdf = pdf[['gene', 'patient_ID','impact_tmp']]

/projects/da_workspace/software/miniconda/envs/dlpy3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
mdf.head(2)

,gene,patient_ID,impact_tmp
0,SAMD11,HTMCP-03-06-02007,NON_SYNONYMOUS_CODING
1,SAMD11,HTMCP-03-06-02214,NON_SYNONYMOUS_CODING


In [26]:
# concat indel and snvs
rdf = pd.concat([mdf,indf])
rdf.head(2)

,gene,patient_ID,impact_tmp
0,SAMD11,HTMCP-03-06-02007,NON_SYNONYMOUS_CODING
1,SAMD11,HTMCP-03-06-02214,NON_SYNONYMOUS_CODING


In [27]:
rdf = rdf[rdf['patient_ID']!="HTMCP-03-06-02061"]

In [28]:
rdf = rdf.drop_duplicates().groupby(['gene', 'patient_ID']).agg({'impact_tmp': ','.join}).reset_index()
rdf.head(2)

,gene,patient_ID,impact_tmp
0,A1BG,HTMCP-03-06-02179,NON_SYNONYMOUS_CODING
1,A1CF,HTMCP-03-06-02020,NON_SYNONYMOUS_CODING


In [30]:
rdf.impact_tmp.unique()

array(['NON_SYNONYMOUS_CODING', 'STOP_GAINED',
       'STOP_GAINED,NON_SYNONYMOUS_CODING', 'SPLICE_SITE_ACCEPTOR',
       'NON_SYNONYMOUS_CODING,STOP_GAINED',
       'NON_SYNONYMOUS_CODING,STOP_GAINED,SPLICE_SITE_ACCEPTOR',
       'missense_variant', 'NON_SYNONYMOUS_CODING,SPLICE_SITE_DONOR',
       'NON_SYNONYMOUS_CODING,FRAME_SHIFT',
       'SPLICE_SITE_ACCEPTOR,NON_SYNONYMOUS_CODING,STOP_GAINED',
       'inframe_deletion', 'SPLICE_SITE_DONOR', 'FRAME_SHIFT',
       'NON_SYNONYMOUS_CODING,SPLICE_SITE_ACCEPTOR', 'STOP_LOST',
       'NON_SYNONYMOUS_CODING,CODON_CHANGE_PLUS_CODON_DELETION',
       'splice_acceptor_variant+intron_variant', 'frameshift_variant',
       'START_LOST', 'SPLICE_SITE_ACCEPTOR,NON_SYNONYMOUS_CODING',
       'START_LOST,NON_SYNONYMOUS_CODING',
       'SPLICE_SITE_DONOR,NON_SYNONYMOUS_CODING',
       'NON_SYNONYMOUS_CODING,STOP_GAINED,FRAME_SHIFT',
       'CODON_CHANGE_PLUS_CODON_DELETION', 'CODON_DELETION',
       'STOP_GAINED,NON_SYNONYMOUS_CODING,SPLICE_SITE_A

In [31]:
def impact_type(x):
#     print(x)
    xsplit = list(set(x.split(',')))
    type = xsplit[0].upper()
    if len(xsplit) > 1:
        impact = 'Multiple'
    elif len(xsplit) == 1:
         if '+' in xsplit[0]:
             impact = 'Multiple'
         elif type == 'MISSENSE_VARIANT' or type == 'NON_SYNONYMOUS_CODING':
             impact = 'NON_SYNONYMOUS_CODING'
         elif  type == 'SPLICE_SITE_ACCEPTOR' or  type == 'SPLICE_SITE_DONOR':
             impact = 'SPLICE_ACCEPTOR_DONOR'
         elif type == 'STOP_LOST':
             impact = 'STOP_LOST'
         elif type == 'STOP_GAINED':
             impact = 'STOP_GAINED'
         elif  type == 'START_LOST':
             impact = 'START_LOST'
         elif type == 'START_GAINED':
             impact = 'START_GAINED'
         elif type == 'FRAME_SHIFT' or type == 'FRAMESHIFT_VARIANT':
             impact = 'FRAME_SHIFT'
         elif type == 'INFRAME_DELETION' or type == 'CODON_INSERTION' or type == 'CODON_DELETION' or type == 'CODON_CHANGE_PLUS_CODON_INSERTION' or type == 'CODON_CHANGE_PLUS_CODON_DELETION' or type.lower() == 'disruptive_inframe_deletion':
             impact = 'CODON_INSERTION_DELETION'                
         else: exit(1);print('ERROR')
    return impact

In [32]:
rdf['impact'] = rdf['impact_tmp'].apply(lambda x: impact_type(x))
rdf = rdf[['gene', 'patient_ID', 'impact']].drop_duplicates()
rdf = rdf.set_index(['gene', 'patient_ID'])['impact'].unstack()
rdf.head(2)

patient_ID,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02007,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02026,...,HTMCP-03-06-02424,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02434,HTMCP-03-06-02435,HTMCP-03-06-02437,HTMCP-03-06-02441,HTMCP-03-06-02442,HTMCP-03-06-02447,HTMCP-03-06-02448
gene,,,,,,,,,,,,,,,,,,,,,
A1BG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A1CF,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NON_SYNONYMOUS_CODING,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [33]:
rdf = pd.concat([rdf, edf])

In [38]:
edf.head()

patient,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02007,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02026,...,HTMCP-03-06-02424,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02434,HTMCP-03-06-02435,HTMCP-03-06-02437,HTMCP-03-06-02441,HTMCP-03-06-02442,HTMCP-03-06-02447,HTMCP-03-06-02448
HIV_status,Positive,Negative,Positive,Negative,Negative,Negative,Negative,Negative,Positive,Negative,...,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive
Putative_histology,Squamous,Squamous,Squamous,Squamous,Adenosquamous,Squamous,Squamous,Squamous,Squamous,Adeno,...,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous,Adeno


In [37]:
rdf.index

Index(['A1BG', 'A1CF', 'A2LD1', 'A2M', 'A2ML1', 'A4GALT', 'A4GNT', 'AAAS',
       'AACS', 'AADAC',
       ...
       'ZXDC', 'ZYG11A', 'ZYG11B', 'ZZEF1', 'ZZZ3', 'hsa-mir-3187',
       'hsa-mir-4763', 'yR211F11.2', 'HIV_status', 'Putative_histology'],
      dtype='object', length=14587)

In [39]:
genes = [ 'HIV_status', 'Putative_histology'] + genes

In [40]:
rdf = rdf.reindex(genes)#.loc['PIK3CA'].value_counts()
rdf.head(3)
pd.Series(rdf.values.ravel()).unique()

array(['Positive', 'Negative', 'Squamous', 'Adenosquamous', 'Adeno',
       'Neuroendocrine', nan, 'NON_SYNONYMOUS_CODING', 'FRAME_SHIFT',
       'Multiple', 'STOP_GAINED', 'CODON_INSERTION_DELETION'],
      dtype=object)

In [28]:
rdf['occurrence'] = (rdf.notnull().sum(axis=1)/(rdf.shape[1] -2))

In [29]:
# genes = rdf['occurrence'].sort_values(ascending=False).index.tolist()
# predetermined gene order

In [30]:
rdf.head()

,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02007,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02026,...,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02434,HTMCP-03-06-02435,HTMCP-03-06-02437,HTMCP-03-06-02441,HTMCP-03-06-02442,HTMCP-03-06-02447,HTMCP-03-06-02448,occurrence
HIV_status,Positive,Negative,Positive,Negative,Negative,Negative,Negative,Negative,Positive,Negative,...,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,1.016529
Putative_histology,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous,Squamous,Squamous,Adeno,...,Squamous,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous,Adeno,1.016529
MLL3,CODON_INSERTION_DELETION,NaN,NaN,NaN,Multiple,NaN,STOP_GAINED,NaN,STOP_GAINED,Multiple,...,NaN,NON_SYNONYMOUS_CODING,NaN,STOP_GAINED,NaN,NON_SYNONYMOUS_CODING,STOP_GAINED,NON_SYNONYMOUS_CODING,NaN,0.181818
MLL2,NaN,STOP_GAINED,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,FRAME_SHIFT,NaN,NaN,NaN,NaN,NaN,0.132231
CREBBP,NaN,NaN,NaN,NaN,NON_SYNONYMOUS_CODING,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.082645


In [31]:
(rdf.loc['MLL3',:] == 'STOP_GAINED').sum()

8

In [32]:
# rdf = rdf.reindex(genes)
# rdf.head(2)

In [33]:
rdf['percentage'] = ['{0}({1}%)'.format(i[0], int(round(i[1]*100))) for i in zip(rdf.index.tolist(), rdf.occurrence)]

In [34]:
rdf.loc['HIV_status', 'percentage'] = 'HIV_Status'
rdf.loc['Putative_histology', 'percentage' ] = 'Puatative_Histology'

In [35]:
("HTMCP-03-06-02061" in rdf.columns)

False

In [36]:
rdf.head(3)

,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02007,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02026,...,HTMCP-03-06-02428,HTMCP-03-06-02434,HTMCP-03-06-02435,HTMCP-03-06-02437,HTMCP-03-06-02441,HTMCP-03-06-02442,HTMCP-03-06-02447,HTMCP-03-06-02448,occurrence,percentage
HIV_status,Positive,Negative,Positive,Negative,Negative,Negative,Negative,Negative,Positive,Negative,...,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,1.016529,HIV_Status
Putative_histology,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous,Squamous,Squamous,Adeno,...,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous,Adeno,1.016529,Puatative_Histology
MLL3,CODON_INSERTION_DELETION,NaN,NaN,NaN,Multiple,NaN,STOP_GAINED,NaN,STOP_GAINED,Multiple,...,NON_SYNONYMOUS_CODING,NaN,STOP_GAINED,NaN,NON_SYNONYMOUS_CODING,STOP_GAINED,NON_SYNONYMOUS_CODING,NaN,0.181818,MLL3(18%)


In [37]:
sdf = rdf.drop('occurrence', axis=1).set_index('percentage', drop=True)

In [38]:
# sdf = sdf.fillna(0)

In [39]:
sdf.head(3)

,HTMCP-03-06-02001,HTMCP-03-06-02002,HTMCP-03-06-02003,HTMCP-03-06-02006,HTMCP-03-06-02007,HTMCP-03-06-02008,HTMCP-03-06-02012,HTMCP-03-06-02013,HTMCP-03-06-02020,HTMCP-03-06-02026,...,HTMCP-03-06-02424,HTMCP-03-06-02427,HTMCP-03-06-02428,HTMCP-03-06-02434,HTMCP-03-06-02435,HTMCP-03-06-02437,HTMCP-03-06-02441,HTMCP-03-06-02442,HTMCP-03-06-02447,HTMCP-03-06-02448
percentage,,,,,,,,,,,,,,,,,,,,,
HIV_Status,Positive,Negative,Positive,Negative,Negative,Negative,Negative,Negative,Positive,Negative,...,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive,Positive
Puatative_Histology,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous,Squamous,Squamous,Adeno,...,Squamous,Squamous,Squamous,Squamous,Squamous,Squamous,Adeno,Squamous,Squamous,Adeno
MLL3(18%),CODON_INSERTION_DELETION,NaN,NaN,NaN,Multiple,NaN,STOP_GAINED,NaN,STOP_GAINED,Multiple,...,NON_SYNONYMOUS_CODING,NaN,NON_SYNONYMOUS_CODING,NaN,STOP_GAINED,NaN,NON_SYNONYMOUS_CODING,STOP_GAINED,NON_SYNONYMOUS_CODING,NaN


In [40]:
sdf.shape[1]

123

In [41]:
assert ("HTMCP-03-06-02061" in sdf.columns) == False

In [42]:
sdf.to_csv('/projects/trans_scratch/validations/workspace/szong/Cervical/variant_bwamem/124_patients_new/histone_mutations_for_oncoprint_123_patients.txt')